<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/maxbergmark/Lab-2/maxbergmark_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Direct methods**
**Max Bergmark**

# **Abstract**

This lab is about implementing QR factorization of matrices, and using the factorization to solve matrix equations 

#**About the code**

I am the author of the code in its entirety. _Add links here_

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import numpy as np

# **Introduction**

# **Methods**

## Helper functions

To improve readability of the code, I implemented these two helper functions.

In [0]:
# return a normalized version of a vector
def normalize(v):
    return v / np.linalg.norm(v)
  
# return the length of a vector
def norm(v):
    return np.linalg.norm(v)

# checks if a matrix is diagonal
def is_diagonal(A):
    return np.allclose(A, np.diag(np.diag(A)))

# checks if a matrix is upper triangular
def is_upper_triangular(A):
    n = A.shape[0]
    for row in range(n):
        if not np.isclose(A[row,:row].sum(), 0, atol = 1e-15):
            return False
    return True

## 1: QR factorization

To get QR factorization working, I implemented a Python version of the pseudo code in Algorithm 3 in section 5.2 (page 70). Since that algorithm gives us both Q and R right away, it was suitable for this task.

In [0]:
def factorize_qr(A):
    n = A.shape[0]
    V = A.copy()
    Q = np.zeros_like(A, dtype = np.float64)
    R = Q.copy()
    for i in range(A.shape[0]):
        r_ii = norm(V[:,i])
        R[i,i] = r_ii
        q_i = V[:,i] / r_ii
        for j in range(i+1, n):
            r_ij = np.dot(q_i, V[:,j])
            V[:,j] -= r_ij * q_i
            R[i,j] = r_ij
        Q[:,i] = q_i
    return Q, R


## 2: direct solver $Ax=b$

With QR factorization working, it is possible to solve equation systems by using Algorithm 2 in section 5.1 (page 69). Since we can rewrite $A = QR$ with $Q$ orthogonal and $R$ upper triangular, we know that $Ax = QRx = b \iff Q^{-1}QRx = Q^{-1}b$, but since $Q$ is orthogonal we also know that $Q^{-1} = Q^T$, which gives us the new form $Rx = Q^Tb$. Since $R$ is upper triangular, we are able to solve the system by first solving for $x_n$, and then move upwards along the rows, substituting the values that are already solved. 

In [0]:
def solve_QR(A, b):
    n = A.shape[0]
    Q, R = factorize_qr(A)
    b_q = np.dot(Q.T, b)
    x = np.zeros_like(b)
    for j in range(n-1, -1, -1):
        x_sum = np.dot(R[j, j+1:], x[j+1:])
        x[j] = (b_q[j] - x_sum) / R[j,j]
    return x

## 3: Least squares problem $Ax=b$

In order to solve over-determined matrix systems, where there are more equations than there are variables, we transform the equation by multiplying both sides by $A^T$, giving us $A^TAx = A^Tb$. Since $A^TA$ is a square matrix, we can use the same algorithm as we did before.

In [0]:
def solve_least_squares(A, b):
    new_A = np.dot(A.T, A)
    new_b = np.dot(A.T, b)
    return solve_QR(new_A, new_b)

## 4: QR eigenvalue algorithm

The algorithm described for generating eigenvalues and corresponding eigenvectors is the repeated QR factorization to create a Schur factorization. The idea is to repeatedly generate $Q_iR_i = A_i$, and then updating $A$ to be $A_{i+1} = R_iQ_i$. Note the order of the multiplicands. 

This repeated iteration will converge to a Schur factorization for many matrices, but more importantly it will converge to a diagonalization if the matrix $A$ is symmetric. If it converges to a diagonal matrix, the eigenvalues of the original $A$ matrix can be found along the diagonal of its iterated counterpart. To get the eigenvectors, you should calculate the matrix product $Q_{eig} = \prod_i Q_i$. The eigenvectors are then found as the column vectors of $Q_{eig}$.

In [0]:
def get_eigenvalues(A):
    A_c = A.copy()
    n = A.shape[0]
    pQ = np.eye(n)
    while not is_upper_triangular(A_c):
    # while not is_diagonal(A_c):
        Q, R = factorize_qr(A_c)
        pQ = np.dot(pQ, Q)
        A_c = np.dot(R, Q)
    eigs = np.diag(A_c)
    return eigs, pQ

# **Results**

## 1: QR factorization

In [0]:
def test_QR_decomposition():
    n = 5
    A = np.random.rand(n, n)
    Q, R = factorize_qr(A)
    # check that R is upper triangular
    for row in range(n):
        assert np.isclose(R[row,:row].sum(), 0)
    # assert that Q is orthogonal
    assert np.allclose(np.dot(Q.T, Q).sum(axis = 1), 1)
    # assert that A = Q*R
    assert np.allclose(A, np.dot(Q, R))
    # assert that Q * Q.T is the identity matrix
    assert np.allclose(np.eye(n), np.dot(Q, Q.T))
    
test_QR_decomposition()
print("Test passed!")

Test passed!


## 2: direct solver $Ax=b$


In [0]:
def test_QR_solve():
    A = np.array([[1, 2], [3, 4]], dtype = np.float64)
    x = np.array([5, 6])
    b = np.dot(A, x)
    x_test = solve_QR(A, b)
    assert np.allclose(x, x_test)

test_QR_solve()
print("Test passed!")

Test passed!


## 3: Least squares problem $Ax=b$

In [0]:
def test_least_squares():
    A = np.array([[1, -1], [1, 1], [2, 1]], dtype = np.float64)
    b = np.array([2, 4, 8])
    x_true = np.array([23/7, 8/7])
    x_test = solve_least_squares(A, b)
    assert np.allclose(x_true, x_test)
    
test_least_squares()
print("Test passed!")

Test passed!


## 4: QR eigenvalue algorithm

In [0]:
def test_eigenvalues(A):
    eig_val, eig_vec = get_eigenvalues(A)
    n = A.shape[0]
    for i in range(n):
        v0 = eig_val[i] * eig_vec[:,i]
        v1 = np.dot(A, eig_vec[:,i])
        assert np.allclose(v0, v1)

def test_all_eigen():
    test_eigenvalues(np.array([[0, -1], [-1, -3]], dtype = np.float64))
    # test_eigenvalues(np.array([[1, 2, 0], [-2, 1, 3], [0, -3, 1]], dtype = np.float64))
    for n in range(2, 9):
        A = np.random.rand(n, n)
        test_eigenvalues(A + A.T)
        
test_all_eigen()
print("Test passed!")

Test passed!


## Full test suite

Use this to run all tests at once.

In [0]:
def run_test_suite():
    test_QR_decomposition()
    test_QR_solve()
    test_least_squares()
    test_all_eigen()

run_test_suite()
print("All tests passed!")

All tests passed!


# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report? 